In [1]:
# default_exp starter

# Ought Text Classification Project

> Binary classification on scientific paper abstracts

In [2]:
#hide
from nbdev.showdoc import *

## GPT-2 Prompt Manipulation

The simplest approach is to to feed in a small number of trainging examples into the prompt directly for zero-shot classification.

### Setup

In [3]:
#export
from transformers import GPT2Tokenizer, GPT2LMHeadModel
import torch
import json
import pandas as pd

In [4]:
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
tokenizer.pad_token = tokenizer.eos_token
model = GPT2LMHeadModel.from_pretrained('gpt2')
model.eval().cuda()

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0): Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
      (1): Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): Laye

### Utilities

In [5]:
#export
def generate(prompt, max_length=5, stop_token=None):
    input_ids = tokenizer.encode(prompt, return_tensors="pt")
    generated_text_ids = model.generate(input_ids=input_ids.cuda(), max_length=max_length+len(input_ids[0]), do_sample=False)
    generated_text = tokenizer.decode(generated_text_ids[0], clean_up_tokenization_spaces=True)
    post_prompt_text = generated_text[len(tokenizer.decode(input_ids[0], clean_up_tokenization_spaces=True)):]
    return prompt + post_prompt_text[:post_prompt_text.find(stop_token) if stop_token else None]

In [6]:
# Note that the logits are shifted over 1 to the left, since HuggingFace doesn't give a logit for the first token
def get_logits_and_tokens(text):
    input_ids = tokenizer.encode(text, return_tensors="pt")
    tokens = [tokenizer.decode([input_id]) for input_id in input_ids[0]]
    output = model(input_ids.cuda())
    return output.logits[0][:-1], tokens

In [7]:
EXAMPLE_PROMPT = """Horrible: negative
Great: positive
Bad:"""

generated_text = generate(EXAMPLE_PROMPT, stop_token="\n")
generated_text

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


'Horrible: negative\nGreat: positive\nBad: negative'

In [8]:
logits, tokens = get_logits_and_tokens(generated_text)
last_token_probs = torch.softmax(logits[-1], dim=0)
negative_prob = last_token_probs[tokenizer.encode(" negative")[0]]
positive_prob = last_token_probs[tokenizer.encode(" positive")[0]]

print(f"tokens: {tokens}\nnegative prob: {negative_prob}\npositive prob: {positive_prob}")

tokens: ['Hor', 'rible', ':', ' negative', '\n', 'Great', ':', ' positive', '\n', 'Bad', ':', ' negative']
negative prob: 0.7252255082130432
positive prob: 0.11788686364889145


### Load Data

Define helper function to load text from `.jsonl` files.

In [9]:
#export
def load_jsonl(filename):
    f = open(filename)
    return [json.loads(line) for line in f.read().splitlines()]

In [10]:
train_examples = load_jsonl("data/train.jsonl")
train_examples[-1]

{'label': 'False',
 'text': 'non relativistic approach for cosmological scalar field dark matter. we derive non relativistic equations of motion for the formation of cosmological structure in a scalar field dark matter sfdm model corresponding to a complex scalar field endowed with a quadratic scalar potential. starting with the full equations of motion written in the newtonian gauge of scalar perturbations we separate out the fields involved into relativistic and non relativistic parts and find the equations of motion for the latter that can be used to build up the full solution. one important assumption will also be that the sfdm field is in the regime of fast oscillations under which its behavior is exactly that of cold dark matter. the resultant equations are quite similar to the schr odinger poisson system of newtonian boson stars plus relativistic leftovers. we exploit that similarity to show how to simulate with minimum numerical effort the formation of cosmological structure in

In [11]:
#export
def render_example(example):
    title = example["text"].split(".")[0].strip()
    abstract = example["text"][len(title)+1:].strip()
    return f'TITLE: {title}\nABSTRACT: {abstract}\nLABEL: {"AI" if example["label"] == "True" else "NOT AI"}'

In [12]:
#export
def render_end_example(example):
    title = example["text"].split(".")[0].strip()
    abstract = example["text"][len(title)+1:].strip()
    return f"TITLE: {title}\nABSTRACT: {abstract}\nLABEL:"

In [13]:
#export
def make_prompt(instructions, train_examples, end_example):
    rendered_train_examples = "\n\n--\n\n".join([render_example(example) for example in train_examples])
    return f"""{instructions}

{rendered_train_examples}

--

{render_end_example(end_example)}"""

In [14]:
INSTRUCTIONS = "Classify the following examples based on whether they are AI-relevant or not:"

prompt = make_prompt(INSTRUCTIONS, train_examples[:4], train_examples[4])
print(prompt)

Classify the following examples based on whether they are AI-relevant or not:

TITLE: thermodynamic analysis of quantum error correcting engines
ABSTRACT: quantum error correcting codes can be cast in a way which is strikingly similar to a quantum heat engine undergoing an otto cycle. in this paper we strengthen this connection further by carrying out a complete assessment of the thermodynamic properties of strokes operator based error correcting codes. this includes an expression for the entropy production in the cycle which as we show contains clear contributions stemming from the different sources of irreversibility. to illustrate our results we study a classical qubit error correcting code well suited for incoherent states and the qubit shor code capable of handling fully quantum states. we show that the work cost associated with the correction gate is directly associated with the heat introduced by the error. moreover the work cost associated with encoding decoding quantum informa

In [15]:
generated_text = generate(prompt, stop_token="\n")
print(generated_text)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Classify the following examples based on whether they are AI-relevant or not:

TITLE: thermodynamic analysis of quantum error correcting engines
ABSTRACT: quantum error correcting codes can be cast in a way which is strikingly similar to a quantum heat engine undergoing an otto cycle. in this paper we strengthen this connection further by carrying out a complete assessment of the thermodynamic properties of strokes operator based error correcting codes. this includes an expression for the entropy production in the cycle which as we show contains clear contributions stemming from the different sources of irreversibility. to illustrate our results we study a classical qubit error correcting code well suited for incoherent states and the qubit shor code capable of handling fully quantum states. we show that the work cost associated with the correction gate is directly associated with the heat introduced by the error. moreover the work cost associated with encoding decoding quantum informa

## Extra Helper Function

These are some extra utility functions that were not defined in the original starter code, but were still useful.

In [3]:
train = load_jsonl("data/train.jsonl")
df = pd.DataFrame(train)

Collect some random samples across classes. This should be flexible enough to generalize beyong the `'AI'` and `'Not AI'` labels.

In [4]:
df.label.unique()

array(['False', 'True'], dtype=object)

In [20]:
samples_per_label = 2
samples = []
for label in df['label'].unique():
    group = df[df.label == label]
    idxs = group.index[:samples_per_label]
    samples += [train[idx] for idx in idxs]
samples

[{'label': 'False',
  'text': 'thermodynamic analysis of quantum error correcting engines. quantum error correcting codes can be cast in a way which is strikingly similar to a quantum heat engine undergoing an otto cycle. in this paper we strengthen this connection further by carrying out a complete assessment of the thermodynamic properties of strokes operator based error correcting codes. this includes an expression for the entropy production in the cycle which as we show contains clear contributions stemming from the different sources of irreversibility. to illustrate our results we study a classical qubit error correcting code well suited for incoherent states and the qubit shor code capable of handling fully quantum states. we show that the work cost associated with the correction gate is directly associated with the heat introduced by the error. moreover the work cost associated with encoding decoding quantum information is always positive a fact which is related to the intrinsic

In [ ]:
#export
def uniform_samples(json='data/train.jsonl', n_samples=2):
    superset = load_jsonl(json)
    df = pd.DataFrame(superset)
    samples = []
    for label in df['label'].unique():
        group = df[df.label == label]
        idxs = group.index[:n_samples]
        samples += [superset[idx] for idx in idxs]
    return samples